# Advanced Document Analytics

In the first three modules, we learned how to represent documents, performed clustering for identifying underlying structure in the documents, and applied classifiers to annotating the documents. In this module we will discuss on regular expressions for cleaning and extracting features from messy text, different language models including n-gram and topic models, and perform sentiment analysis on text. 




## Phrase Modeling

TF and TFIDF representations treat each text document as a vector of terms, where terms are the basic units/features. In real-world examples, we observe co-occurrences of words (i.e. phrases). E.g., United States, machine learning, data science. Sometimes it is desirable that these co-occurrences of words are treated as features. To capture such phrases, N-gram model is proposed. N-gram is a statistical language model that assigns probabilities to a sequence of words.

You can think of an N-gram as the sequence of N words, by that notion, a 2-gram (or bigram) is a two-word sequence of words like "How are", "are you", or "you doing?", and a 3-gram (or trigram) is a three-word sequence of words like "how are you", or "are you doing?".

Formally, given a history $h = <w_1,\dots, w_m>$, an N-gram model estimates the conditional probability of a word $w$ given the preceding $n-1$ words of $h$, i.e., $P(w|h) \sim P(w|w_{n-1},\dots,w_{1})$. Consider a bigram model, which approximates the probability of a word given all the previous words by using only the conditional probability of one preceding word. In other words, you approximate it with the probability: P(are | how). 

When we use a bigram model to predict the conditional probability of the next word, we are thus making the following approximation:

$$
P(w_n|w_1, w_2,\dots, w_{n-1}) \sim P(w_n|w_{n-1})
$$

This above approximation is also known as Markov assumption: the future state depends on the current state only and does not require past states.


## Topic Modeling

In computational linguistic, topic modeling methods aim to discover topics in a corpus in an unsupervised setting. A topic consist of a set of words. Unlike hard clustering, a word may belong to multiple topics. **Formally, a topic is a probability distribution over words (or vocabulary)**. Topic models essentially represent **a document as a mixture of topics** and **a topic as a probability distribution over words**. An input to the topic model is a term frequency matrix. 

Let's understand the concept of topic modeling with a toy example. Suppose we have a corpus of news articles which is collected from the politics, sports, and health section of a news portal. In a fictitious analysis we came up with a vocabulary consisting of 10 words: budget, protest, election, contest, football, NFL, Olympic, corona, vaccine, mask. Given the corpus (without any annotations), a topic modeling method might give us following three distributions (or topics): 

Dist   |budget|protest|election|contest|football|NFL|Olympic|corona|vaccine|mask|
-------|------|-------|--------|-------|--------|---|-------|------|-------|----|
Dist 1 |  0.10|   0.20|    0.40|   0.15|    0.00|0.00|  0.00|0.07  | 0.05  |0.03|
Dist 1 |  0.05|   0.05|    0.00|   0.15|    0.30|0.30|  0.10|0.05  | 0.00  |0.00|
Dist 1 |  0.03|   0.00|    0.00|   0.00|    0.00|0.00|  0.00|0.50  | 0.30  |0.17|

If we set 0.1 as a probability threshold for a word to be meaningful in a topic, then it becomes obvious these three distributions refer to following topics: 

* **Politics**: budget, protest, election, contest
* **Sports**: football, NFL, Olympic
* **Health**: corona, vaccine, mask

**How does topic model represent a document?** Documents are represented as a mixture of topics. E.g., consider a news article in the above toy corpus that discusses the impact corona virus in the US politics. We can assume that both of the topics, Politics and Health, should have representations in this news article. E.g., this article can be represented as a mixture of 60% Politics and 40% Health; so, in this article words from the Politics topic would be more visible compared to the words from the Health topic. Note that here a document also become a bag of words (i.e., the ordering between words is absent).


### Latent Dirichlet Allocation

A well-known method for topic modeling is **Latent Dirichlet Allocation (LDA)**, proposed by [Blei et al.](https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) (2003). It is a generative  and an unsupervised model. The key idea is that if we can define a model that can generate synthetic documents given the model parameters, then we can fit the  model parameters given a set of real-world documents. E.g., Given the above three topics (i.e., distributions over words), we can generate a news article as follows: 

- Sample a topic mixture ($\theta_d$) for a document $d$ (e.g., 60% Sports, 20% Politics, and 20% Health)
    - $\theta_d \sim \text{Dir}(\alpha) $ (Dir stands for Dirichlet distribution)
- Sample the number of words ($N_d$) for the document $d$
    - $N_d \sim \text{Poisson}(\gamma)$ 
- For generating each ($i$-th) of the $N_d$ words 
    - Sample a topic ($z_i$) for the word $w_i$
        - $ z_i \sim \text{Multinomial}(\theta)$
    - Given the topic ($z_i$) of the word, sample the word
        - $ w_i \sim \text{Multinomial}(z_i)$

The name LDA came from the fact that this method samples topic mixture from a Dirichlet distribution, which is a multivariate version of beta distribution. A beta distribution gives a value in [0, 1] and a Dirichlet distribution gives a vector of values in [0, 1] and sum of the vector is 1.  

The above generative model can be represented as a probabilistic graphical models (see below). We are skipping the discussion on graphical models as this topic is beyond the scope of this course. 

<img src="../images/lda.png" alt="Simply Easy Learning" width="600"
         height="400">

Here the boxes are "plates" representing replicates.
The left outer plate represents documents, while the left inner plate represents the repeated choice
of topics and words within a document. Programmatically we can think of each plate (box) as a for loop. So the left outer and inner box represents a double for loop: 

* The left outer plate repeats D number of times (i.e., one for each document)
* For each of document, the inner plate repeats N times (i.e., one for each word)

The right box represents $K$ topics. All the circles here are random variable; filled and unfilled circles denote observed and unobserved, variables, respectively. The fact that topic models only consider words as observed variable and the rest are unobserved makes these methods powerful and popular.

#### Additional Reading
* https://scikit-learn.org/stable/modules/decomposition.html#latent-dirichlet-allocation-lda
* https://cfss.uchicago.edu/notes/topic-modeling/ (in R)

### Non-negative Matrix Factorization

In the LDA method, given a term frequency matrix $X$, we see that a document is represented with $K$ topic dimensions and each topic has $W$ term dimensions, which is the size of the vocabulary. We can perform similar decomposition of a corpus (i.e all documents) using a Matrix Factorization. 

Let's assume $X$ has a dimension $N \times F$, where $N$ is the number of documents and $F$ is the number of features (i.e, vocabulary size). If we want to extract $K$ topics, then we need to factorize X into two non-negative matrices $W$ and $H$, with a constraint that $W$ has a dimension $N \times K$ and $H$ has a dimension $K \times F$. In practice exact factorization is difficult; so methods aim to find a approximate a solution by minimizing the following objective function:

$$
\frac{1}{2} \left\| X - WH \right\|_{Fro}^2
$$

Here [Frobineus Norm](https://en.wikipedia.org/wiki/Matrix_norm#Frobenius_norm) is used to measure the distance between $X$ and $WH$. As you can see $W$ (shape $N \times K$) is a matrix for $K$ topic mixtures for $N$ documents,  and $H$ is a matrix for topic distributions for $K$ topics. 


#### Additional Reading
https://scikit-learn.org/stable/modules/decomposition.html#nmf

## Sentiment Analysis

Sentiment analysis is the process of determining the emotional tone behind a series of words, used to gain an understanding of the the attitudes, opinions and emotions expressed within the text. Sometimes sentiment analysis is also called opinion mining. 

Sentiment analysis is extremely useful in social media monitoring as it allows us to gain an overview of the wider public opinion behind certain topics. As this information is subjective, it helps business to understand the social sentiment of their brand, product or service. Sentiment Analysis is also useful for practitioners and researchers, especially in fields like sociology, marketing, advertising, psychology, economics, and political science, which rely a lot on human-computer interaction data.

Usually sentiments within a text is expressed in terms polarity such as positive, negative, or neutral. It is also possible to introduce more finer grain polarities in the text. See the following two article to learn more about sentiment analysis. 

* https://theappsolutions.com/blog/development/sentiment-analysis/#contents_7
* https://monkeylearn.com/sentiment-analysis/